In [1]:
# Import our dependencies

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

# Import our input dataset

attrition_df = pd.read_csv('HR-Employee-Attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [2]:
# Generate our categorical variable list

attrition_cat = attrition_df.dtypes[attrition_df.dtypes == "object"].index.tolist()

In [3]:
# Check the number of unique values in each column

attrition_df[attrition_cat].nunique()

Attrition         2
BusinessTravel    3
Department        3
EducationField    6
Gender            2
JobRole           9
MaritalStatus     3
Over18            1
OverTime          2
dtype: int64

In [4]:
# Create a OneHotEncoder instance

enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list

encode_df = pd.DataFrame(enc.fit_transform(attrition_df[attrition_cat]))

# Add the encoded variable names to the DataFrame

encode_df.columns = enc.get_feature_names(attrition_cat)
encode_df.head()

C:\Users\Nick\anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Attrition_No,Attrition_Yes,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Human Resources,Department_Research & Development,Department_Sales,EducationField_Human Resources,EducationField_Life Sciences,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
3,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


In [5]:
# Merge one-hot encoded features and drop the originals

attrition_df = attrition_df.merge(encode_df,left_index=True, right_index=True)
attrition_df = attrition_df.drop(attrition_cat,1)
attrition_df.head()

C:\Users\Nick\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  after removing the cwd from sys.path.


,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
0,41,1102,1,2,1,1,2,94,3,2,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1,49,279,8,1,1,2,3,61,2,2,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,37,1373,2,2,1,4,4,92,2,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
3,33,1392,3,4,1,5,4,56,3,1,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,27,591,2,1,1,7,1,40,3,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


In [6]:
# Split our preprocessed data into our features and target arrays

y = attrition_df["Attrition_Yes"].values
X = attrition_df.drop(["Attrition_Yes","Attrition_No"],1).values

# Split the preprocessed data into a training and testing dataset

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

C:\Users\Nick\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  after removing the cwd from sys.path.


In [7]:
# Create a StandardScaler instance

scaler = StandardScaler()

# Fit the StandardScaler

X_scaler = scaler.fit(X_train)

# Scale the data

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [8]:
# Define the model - deep neural net

number_input_features = len(X_train[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer

nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer

nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer

nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model

nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 448       
                                                                 
 dense_1 (Dense)             (None, 5)                 45        
                                                                 
 dense_2 (Dense)             (None, 1)                 6         
                                                                 
Total params: 499
Trainable params: 499
Non-trainable params: 0
_________________________________________________________________


In [9]:
# Compile the model

nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [10]:
# Train the model

fit_model = nn.fit(X_train,y_train,epochs=100)

Epoch 1/100
35/35 [==============================] - 1s 913us/step - loss: 1966.0436 - accuracy: 0.1779
Epoch 2/100
35/35 [==============================] - 0s 795us/step - loss: 517.7761 - accuracy: 0.4419
Epoch 3/100
35/35 [==============================] - 0s 795us/step - loss: 113.7837 - accuracy: 0.7341
Epoch 4/100
35/35 [==============================] - 0s 795us/step - loss: 90.1817 - accuracy: 0.7922
Epoch 5/100
35/35 [==============================] - 0s 795us/step - loss: 76.8644 - accuracy: 0.7858
Epoch 6/100
35/35 [==============================] - 0s 795us/step - loss: 63.7390 - accuracy: 0.7813
Epoch 7/100
35/35 [==============================] - 0s 795us/step - loss: 50.7326 - accuracy: 0.7877
Epoch 8/100
35/35 [==============================] - 0s 795us/step - loss: 37.6509 - accuracy: 0.7868
Epoch 9/100
35/35 [==============================] - 0s 824us/step - loss: 24.5194 - accuracy: 0.7858
Epoch 10/100
35/35 [==============================] - 0s 824us/step - loss: 16

35/35 [==============================] - 0s 824us/step - loss: 1.1311 - accuracy: 0.7877
Epoch 82/100
35/35 [==============================] - 0s 824us/step - loss: 0.9385 - accuracy: 0.7904
Epoch 83/100
35/35 [==============================] - 0s 795us/step - loss: 1.3371 - accuracy: 0.7668
Epoch 84/100
35/35 [==============================] - 0s 795us/step - loss: 1.4657 - accuracy: 0.7868
Epoch 85/100
35/35 [==============================] - 0s 824us/step - loss: 0.7893 - accuracy: 0.7931
Epoch 86/100
35/35 [==============================] - 0s 795us/step - loss: 2.0086 - accuracy: 0.7804
Epoch 87/100
35/35 [==============================] - 0s 795us/step - loss: 0.9386 - accuracy: 0.7904
Epoch 88/100
35/35 [==============================] - 0s 765us/step - loss: 1.0093 - accuracy: 0.7868
Epoch 89/100
35/35 [==============================] - 0s 795us/step - loss: 1.8220 - accuracy: 0.7858
Epoch 90/100
35/35 [==============================] - 0s 795us/step - loss: 1.2548 - accuracy: 

In [11]:
# Evaluate the model using the test data

model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

12/12 - 0s - loss: 0.7566 - accuracy: 0.8668 - 115ms/epoch - 10ms/step
Loss: 0.7565868496894836, Accuracy: 0.866847813129425


In [12]:
# Import checkpoint dependencies

import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames

os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [13]:
# Compile the model

nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every epoch

cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch')

# Train the model

fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

# Evaluate the model using the test data

model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
 1/35 [..............................] - ETA: 15s - loss: 0.8569 - accuracy: 0.5312
Epoch 1: saving model to checkpoints\weights.01.hdf5
35/35 [==============================] - 1s 1ms/step - loss: 0.7438 - accuracy: 0.6162
Epoch 2/100
 1/35 [..............................] - ETA: 0s - loss: 0.4130 - accuracy: 0.7812
Epoch 2: saving model to checkpoints\weights.02.hdf5
35/35 [==============================] - 0s 1ms/step - loss: 0.5405 - accuracy: 0.6887
Epoch 3/100
 1/35 [..............................] - ETA: 0s - loss: 0.4210 - accuracy: 0.7500
Epoch 3: saving model to checkpoints\weights.03.hdf5
35/35 [==============================] - 0s 1ms/step - loss: 0.4688 - accuracy: 0.7831
Epoch 4/100
 1/35 [..............................] - ETA: 0s - loss: 0.4149 - accuracy: 0.8438
Epoch 4: saving model to checkpoints\weights.04.hdf5
35/35 [==============================] - 0s 971us/step - loss: 0.4477 - accuracy: 0.8058
Epoch 5/100
 1/35 [..............................] - ETA:

35/35 [==============================] - 0s 971us/step - loss: 0.2791 - accuracy: 0.8893
Epoch 36/100
 1/35 [..............................] - ETA: 0s - loss: 0.1746 - accuracy: 0.9375
Epoch 36: saving model to checkpoints\weights.36.hdf5
35/35 [==============================] - 0s 1ms/step - loss: 0.2772 - accuracy: 0.8902
Epoch 37/100
 1/35 [..............................] - ETA: 0s - loss: 0.2673 - accuracy: 0.9375
Epoch 37: saving model to checkpoints\weights.37.hdf5
35/35 [==============================] - 0s 972us/step - loss: 0.2747 - accuracy: 0.8893
Epoch 38/100
 1/35 [..............................] - ETA: 0s - loss: 0.3161 - accuracy: 0.8125
Epoch 38: saving model to checkpoints\weights.38.hdf5
35/35 [==============================] - 0s 971us/step - loss: 0.2722 - accuracy: 0.8920
Epoch 39/100
 1/35 [..............................] - ETA: 0s - loss: 0.3007 - accuracy: 0.8750
Epoch 39: saving model to checkpoints\weights.39.hdf5
35/35 [==============================] - 0s 1m

 1/35 [..............................] - ETA: 0s - loss: 0.1959 - accuracy: 0.9062
Epoch 70: saving model to checkpoints\weights.70.hdf5
35/35 [==============================] - 0s 971us/step - loss: 0.2233 - accuracy: 0.9156
Epoch 71/100
 1/35 [..............................] - ETA: 0s - loss: 0.2457 - accuracy: 0.8750
Epoch 71: saving model to checkpoints\weights.71.hdf5
35/35 [==============================] - 0s 942us/step - loss: 0.2228 - accuracy: 0.9156
Epoch 72/100
 1/35 [..............................] - ETA: 0s - loss: 0.1317 - accuracy: 0.9375
Epoch 72: saving model to checkpoints\weights.72.hdf5
35/35 [==============================] - 0s 971us/step - loss: 0.2212 - accuracy: 0.9156
Epoch 73/100
 1/35 [..............................] - ETA: 0s - loss: 0.1580 - accuracy: 0.9375
Epoch 73: saving model to checkpoints\weights.73.hdf5
35/35 [==============================] - 0s 972us/step - loss: 0.2202 - accuracy: 0.9165
Epoch 74/100
 1/35 [..............................] - ETA

In [14]:
# Define the model - deep neural net

number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5

nn_new = tf.keras.models.Sequential()

# First hidden layer

nn_new.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer

nn_new.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer

nn_new.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model

nn_new.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Restore the model weights

nn_new.load_weights("checkpoints/weights.100.hdf5")

# Evaluate the model using the test data

model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

12/12 - 0s - loss: 0.3570 - accuracy: 0.8967 - 133ms/epoch - 11ms/step
Loss: 0.35701122879981995, Accuracy: 0.89673912525177


In [15]:
# Export our model to HDF5 file

nn_new.save("trained_attrition.h5")

In [16]:
# Import the model to a new object

nn_imported = tf.keras.models.load_model('trained_attrition.h5')

In [17]:
# Evaluate the completed model using the test data

model_loss, model_accuracy = nn_imported.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

12/12 - 0s - loss: 0.3570 - accuracy: 0.8967 - 88ms/epoch - 7ms/step
Loss: 0.35701122879981995, Accuracy: 0.89673912525177
